# Tow-Based Discontinuous Composite (TBDC) property predictor
This code is written for an MSc research project at Imperial College London

## Mount drive

In [1]:
# Data to be imported from google drive
from google.colab import drive
drive.mount('/content/drive')

# Set path for drive with datasets
drive_path ="/content/drive/MyDrive/Individual_project_ICL"

Mounted at /content/drive


## Imports and installs

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas

In [3]:
import os
import random
import time
import math
import datetime
import shutil

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Import data


In [45]:
# Read data from txt file in format "parameter:value"\n e.g. N_t:[8, 8]\n
with open(drive_path + '/DatDump.txt') as f:
    dat = f.read()
dat = dat.splitlines() # split into string array by linebreaks

namesToLoad = ['theta_t', 'failure_factors'] # set variables to load
datDict = {} # create dictionary to store data, for easier debugging

for n in namesToLoad: # For each variable to be loaded
    paramStr = [match for match in dat if n+':' in match] # find all string array entries with the given parameter name (use ":" to ensure only the actual variable is taken)
    varname = paramStr[0].split(':')[0] # Set variable name for dictionary
    datDict[varname] = paramStr[0].split(':')[-1] # assign key and value for parameter in dictionary
# np.array()
theta_t = datDict['theta_t'] # Angle of each ply in laminate
failure_factors = datDict['failure_factors'] #

failure_factors = np.array(list(failure_factors), dtype=float)
print(failure_factors)

ValueError: ignored

In [44]:
datDict[varname]

'[308.8, 668.6, 2718.4, 105.8, 554.6, 338.2, 523.6, 130.8, 1516.2, 277.8, 35.8, 250.4, 363.4, 163.0, 694.2, 38.4, 586.2, 56.4, 223.0, 39.4, 702.8, 402.0, 47.4, 49.8, 615.4, 77.4, 770.2, 489.2, 557.4, 436.2, 997.0, 461.6, 36.6, 627.4, 429.8, 27.8, 219.4, 441.6, 38.8, 28.4, 105.2, 38.2, 263.2, 336.8, 51.8, 644.6, 56.8, 122.8, 971.2, 431.6]'